# Compare and benchmark the calculation of dipole matrix elements with pairinteraction and ARC

In [1]:
import matplotlib.pyplot as plt
import numpy as np

from numerov.rydberg import RydbergState
from numerov.units import ureg

In [2]:
# A few exemplary test cases, where pairinteraction and ARC do fail in various ways
n_list = np.arange(20, 150)

choose = "circular"

dn, dl, dj, dm = (3, 1, 0, 0)
# dn, dl, dj, dm = (1, 0, 0, 0)
# dn, dl, dj, dm = (2, 0, 0, 0)
# dn, dl, dj, dm = (2, 2, 2, 0)
# dn, dl, dj, dm = (5, 1, 0, 0)
# dn, dl, dj, dm = (5, 2, 1, 0)

##### Circular states
if choose == "circular":
    qn1_list = [(n1, n1 - 1, n1 - 0.5, n1 - 0.5) for n1 in n_list]
    qn2_list = [(n + dn, l + dl, j + dj, m + dm) for n, l, j, m in qn1_list]

###### Other states
if choose == "close_to_circular":
    dl1 = 5
    qn1_list = [(n1, n1 - dl1, n1 - dl1 + 0.5, n1 - dl1 + 0.5) for n1 in n_list]
    qn2_list = [(n + dn, l + dl, j + dj, m + dm) for n, l, j, m in qn1_list]

if choose == "works_fine":
    n_list = list(range(30, 100))
    l1 = 25
    qn1_list = [(n, l1, l1 + 0.5, l1 + 0.5) for n in n_list]
    qn2_list = [(n + dn, l + dl, j + dj, m + dm) for n, l, j, m in qn1_list]

if choose == "sign_error":
    n_list = list(range(7, 80))
    qn1_list = [(n, 0, 0.5, 0.5) for n in n_list]
    qn2_list = [(n + 2, 1, 1.5, 0.5) for n in n_list]

In [3]:
results = {}

In [4]:
matrixelements = []
for qn1, qn2 in zip(qn1_list, qn2_list):
    print(f"n={qn1[0]}", end="\r")
    q = qn2[-1] - qn1[-1]
    state_i = RydbergState("Rb", *qn1)
    state_f = RydbergState("Rb", *qn2)
    dipole_me = state_i.calc_multipole_matrix_element(state_f, 1, 1, q, unit="a.u.")
    matrixelements.append(dipole_me)

results["Numerov"] = np.array(matrixelements)

In [5]:
import os

from pairinteraction import pireal as pi

os.makedirs("./.pairinteraction_cache/", exist_ok=True)
cache = pi.MatrixElementCache("./.pairinteraction_cache/")

cache.setMethod(pi.NUMEROV)
matrixelements = []
for qn1, qn2 in zip(qn1_list, qn2_list):
    print(f"n={qn1[0]}", end="\r")
    state_i = pi.StateOne("Rb", int(qn1[0]), int(qn1[1]), qn1[2], qn1[3])
    state_f = pi.StateOne("Rb", int(qn2[0]), int(qn2[1]), qn2[2], qn2[3])

    dipole_me = cache.getElectricMultipole(state_i, state_f, 1, 1)
    matrixelements.append(dipole_me)

pi_unit_to_au = (ureg.Quantity(1, "cm/V") * ureg.Quantity(1, "GHz").to("J", "spectroscopy")).to("e * a_0").magnitude
matrixelements = np.array(matrixelements) * pi_unit_to_au

results["Pairinteraction Numerov"] = matrixelements

In [6]:
import arc

atom = arc.Rubidium87()

matrixelements = []
for qn1, qn2 in zip(qn1_list, qn2_list):
    print(f"n={qn1[0]}", end="\r")
    q = int(qn2[-1] - qn1[-1])
    v = atom.getDipoleMatrixElement(
        int(qn1[0]),
        int(qn1[1]),
        float(qn1[2]),
        float(qn1[3]),
        int(qn2[0]),
        int(qn2[1]),
        float(qn2[2]),
        float(qn2[3]),
        q=int(q),
    )
    matrixelements.append(v)

results["ARC"] = np.array(matrixelements)

Numerov 0.5206962455689754
Pairinteraction Numerov 0.49158978222787036
ARC 0.6749529939761194


In [7]:
ls_dict = {"Pairinteraction Numerov": "--", "ARC": ":"}

fig, ax = plt.subplots()

for key, values in results.items():
    ls = ls_dict.get(key, "-")
    ax.plot(n_list, values, ls=ls, label=key)
    print(key, np.max(np.abs(values)))

ax.set_xlabel("n")
ax.set_ylabel(r"Dipole Matrix element [$e \cdot a_0$]")

ax.legend()
plt.show()